## Time a Ligand Polymer Interaction Calculation using Python Implementation

In [ ]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.interactions import InteractionFingerprinter, InteractionFilter
from mmtfPyspark.webfilters import Pisces
import papermill as pm
import time

### Run benchmark

In [ ]:
t0 = time.time()
spark = SparkSession.builder.master("local[" + str(cores) + "]").appName("Fingerprint").getOrCreate()

t1 = time.time()
structures = mmtfReader.read_sequence_file(path)
structures = structures.filter(lambda s: s[1].num_models == 1)
                               
interaction_filter = InteractionFilter()
interaction_filter.set_target_elements(False, ['C','H','P'])
interaction_filter.set_query_elements(True, ['Zn'])
interaction_filter.set_distance_cutoff(3.0)

interactions = InteractionFingerprinter.get_ligand_polymer_interactions(structures, interactionFilter)
count = interactions.count()

t2 = time.time()

spark.stop()
t3 = time.time()

### Record results

In [ ]:
pm.record("interactions_total", t3 - t0)
pm.record("interactions", t2 - t1)
pm.record("count", count)